In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType, LongType, TimestampType 
from pyspark.sql.functions import concat_ws, col, regexp_replace, when, lit, desc, max, to_date, regexp_extract, count, to_timestamp
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
import re

In [0]:
%run /Workspace/Repos/felipegoraro@outlook.com/projeto_nintendo/src/config/spark_performance

In [0]:
def carregando_ultima_partition(nome_tabela, coluna_particao):

    schema = StructType([
        StructField("codigo", StringType(), True),
        StructField("origem", StringType(), True),
        StructField("extract", StringType(), True),
        StructField("desconto", StringType(), True),
        StructField("link", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("parcelamento", StringType(), True),
        StructField("preco", StringType(), True),
        StructField("data_ref", StringType(), True)
    ])

    print(f"Tentando ler arquivo Delta de: {nome_tabela}")
    
    spark = SparkSession.builder.getOrCreate()
    dbutils = DBUtils(spark) 

    # 1- Listando arquivos Delta no Volume Bronze
    print(f"\n--- Verificação do caminho Volume Bronze ---")
    try:
        dbutils.fs.ls(nome_tabela)
        print(f"STATUS OK - Caminho '{nome_tabela}' existe. Tentando carregar como Delta.")
    except Exception as e:
        print(f"STATUS ALERTA - O caminho '{nome_tabela}' não existe ou não é acessível. Detalhe: {e}. Gerando um DataFrame vazio.")
        print("\nRetornando um dataframe vazio!")
        # Em caso de erro, retorne um dataframe vazio com o schema definido anteriormente
        return spark.createDataFrame([], schema=schema)
    
    # 2-  Recuperar partições existentes
    try:
        df = spark.read.format("delta").load(nome_tabela)
    except Exception as e:
        print(f"STATUS ALERTA - Erro ao acessar a tabela '{nome_tabela}': {e}")

    try:

        ultima_particao_df = df.select(max(coluna_particao).alias("ultima_particao"))
        
        ultima_particao = (
            ultima_particao_df.first()["ultima_particao"]
            if ultima_particao_df.first()
            else None
        )

    except Exception as e:
        print(f"STATUS ALERTA - Erro ao obter última partição da tabela '{nome_tabela}': {e}")

    # 3- Tentando ler o arquivo Delta do Volume Bronze
    print(f"\n--- Verificação da leitura do arquivo Delta ---")
    try:

        filtro = f"{coluna_particao} = '{ultima_particao}'"

        df = spark.read.format("delta").load(nome_tabela) \
                .where(filtro)
        print(f"STATUS OK - Tabela Delta '{nome_tabela}' carregada com sucesso pela partição: {ultima_particao}.")

    except Exception as e:
        # Em caso de erro, retorne um dataframe vazio com o schema definido anteriormente
        print(f"STATUS ALERTA - Erro ao carregar a tabela Delta '{nome_tabela}'. Provavelmente não é uma tabela Delta válida ou não contém dados. Detalhe do erro: {e}")
        print("\nRetornando um dataframe vazio!")
        return spark.createDataFrame([], schema=schema)
    
    # 4- Verificando se o dataframe está vazio
    # Em caso positivo, retorne um dataframe vazio com o schema definido anteriormente
    print(f"\n--- Verificação se arquivo Delta está vazio ---")
    if df.rdd.isEmpty():
        print(f"STATUS ALERTA - Tabela '{nome_tabela}' foi carregada como Delta VÁLIDA, mas está completamente vazia. Retornando DataFrame vazio com o schema definido.")
        print("\nRetornando um dataframe vazio!")
        return spark.createDataFrame([], schema=schema)
    else:
        print(f"STATUS OK - Tabela Delta '{nome_tabela}' não está vazio.")
        return df


# Caminho para a external location do diretório bronze
bronze_path = f"/Volumes/nintendodatabrickspkjgt7_workspace/nintendo/bronze"

# Lendo arquivo Delta do diretório bronze pela ultima partição
df = carregando_ultima_partition(bronze_path, "data_ref")

In [0]:
def filter_not_null_value(df, coluna):

    print(f"Iniciando o filtro de valores vazios na coluna: {coluna}")

    dffiltered = df.filter(col(coluna).isNotNull())

    qtddotal = df.count()
    qtdnotnull = df.filter(col(coluna).isNotNull()).count()
    qtdnull = df.filter(col(coluna).isNull()).count()

    print(f"\nDataframe filtrado, numero de linhas: {qtdnotnull}")

    assert qtddotal == (qtdnull + qtdnotnull), \
    f"\nErro na contagem: O total de linhas ({qtddotal}) não é igual à soma de nulos ({qtdnull}) e não nulos ({qtdnotnull}) para a coluna '{coluna}'."

    print(f"\nFiltro ralizado com sucesso")
    print(f"\ndf origem {qtddotal} linhas = df filtrado {qtdnotnull} linhas + df não filtrado {qtdnull} linhas")

    return dffiltered

dffiltered = filter_not_null_value(df, "codigo")

In [0]:
def define_data_columns(df):

    print(f"Iniciando conversão de colunas com padrões de data/timestamp")

    formato_data_regex = r"^\d{4}-\d{2}-\d{2}$"
    formato_timestamp_regex = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}$"

    colunas_string = [coluna for coluna, dtype in df.dtypes if dtype == "string"]

    print(f"\nColunas strings identificadas no dataframe: {colunas_string}")

    for coluna in colunas_string:
        df_sem_nulos = df.filter(col(coluna).isNotNull())

        # Verifica se há dados na coluna para evitar erro em count() em DF vazio
        if df_sem_nulos.count() == 0:
            print(f"Coluna '{coluna}' não possui valores não nulos. Ignorando.")
            continue

        match_data_count = df_sem_nulos.filter(regexp_extract(col(coluna), formato_data_regex, 0) != "").count()
        match_timestamp_count = df_sem_nulos.filter(regexp_extract(col(coluna), formato_timestamp_regex, 0) != "").count()
        total_count = df_sem_nulos.count()

        if match_data_count == total_count:
            print(f"\nColuna '{coluna}' com padrões de data para a conversão (yyyy-MM-dd).")
            df = df.withColumn(coluna, to_date(col(coluna), "yyyy-MM-dd"))
            novo_tipo = dict(df.dtypes)[coluna]
            assert novo_tipo == "date", f"STATUS ALERTA -  A coluna {coluna} não foi convertida corretamente para 'date'! Tipo atual: {novo_tipo}"
            print(f"STATUS OK - Coluna '{coluna}' convertida com sucesso, tipo identificado = {novo_tipo}.")

        elif match_timestamp_count == total_count:
            print(f"\nColuna '{coluna}' com padrões de timestamp para a conversão (yyyy-MM-dd HH:mm).")
            df = df.withColumn(coluna, to_timestamp(col(coluna), "yyyy-MM-dd HH:mm"))
            novo_tipo = dict(df.dtypes)[coluna]
            assert novo_tipo == "timestamp", f"STATUS ALERTA -  A coluna {coluna} não foi convertida corretamente para 'timestamp'! Tipo atual: {novo_tipo}"
            print(f"STATUS OK - Coluna '{coluna}' convertida com sucesso, tipo identificado = {novo_tipo}.")

        else:
            print(f"\nColuna '{coluna}' não corresponde a nenhum padrão de data/timestamp conhecido. Ignorando.")

    return df

df_convert_data = define_data_columns(dffiltered)

In [0]:
def tratar_parcelamento(df):

    print(f"Iniciando o tratamento de coluna referente à 'parcelamento' no dataframe")

    # Extraia o número de parcelas (já tratando nulos)
    df_com_parcelas = df.withColumn(
        "numero_parcelas",
        when(col("parcelamento").isNotNull(), regexp_extract(col("parcelamento"), r'(\d+)x', 1)).otherwise(lit(0))
    )
    df_com_parcelas = df_com_parcelas.withColumn("numero_parcelas", col("numero_parcelas").cast(LongType()))

    # Extraia o valor da prestação (já tratando nulos e convertendo para Double)
    df_com_valores = df_com_parcelas.withColumn(
        "valor_prestacao",
        when(col("parcelamento").isNotNull(), regexp_extract(col("parcelamento"), r'R\$ (\d+,\d{2})', 1)).otherwise(lit("0"))
    ).withColumn(
        "valor_prestacao",
        when(col("valor_prestacao") != '0',
            regexp_replace(col("valor_prestacao"), ",", ".").cast(DoubleType())
        ).otherwise(lit(0.0))
    )

    schema = df_com_valores.schema
    print(f"\nVerificando Schema no dataframe\n")

    if "numero_parcelas" in schema.fieldNames() and schema["numero_parcelas"].dataType == LongType():
        print("STATUS OK - A coluna 'numero_parcelas' foi criada e tem o tipo correto (bigint).")
    else:
        print("STATUS ALERTA - A coluna 'numero_parcelas' não foi criada ou tem o tipo incorreto.")

    if "valor_prestacao" in schema.fieldNames() and schema["valor_prestacao"].dataType == DoubleType():
        print("STATUS OK - A coluna 'valor_prestacao' foi criada e tem o tipo correto (double).")
    else:
        print("STATUS ALERTA - A coluna 'valor_prestacao' não foi criada ou tem o tipo incorreto.")

    # Remova a coluna original "parcelamento"
    df_final = df_com_valores.drop("parcelamento")

    if "parcelamento" not in df_final.columns:
        print("STATUS OK - A coluna 'parcelamento' foi removida com sucesso.")
    else:
        print("STATUS ALERTA -Erro: A coluna 'parcelamento' ainda está presente.")

    return df_final

df_parcelado = tratar_parcelamento(df_convert_data)

In [0]:
def define_numeric_columns(df):

    print(f"Iniciando conversão de colunas com padrões inteiros/decimais")

    # Regex para identificar valores percentuais e monetários
    regex_percentual = re.compile(r"^\d+%$")
    regex_monetario = re.compile(r"^R\$?\s?\d{1,3}(\.\d{3})*(,\d{2})?$")

    # Obtendo colunas de tipo string
    colunas_string = [coluna for coluna, dtype in df.dtypes if dtype == "string"]
    colunas_percentuais = []
    colunas_monetarias = []

    # Identifica colunas com valores percentuais e monetários
    for coluna in colunas_string:
        df_sem_nulos = df.filter(col(coluna).isNotNull())
        valores_amostra = df_sem_nulos.select(coluna).rdd.map(lambda row: row[0]).collect()

        if any(bool(regex_percentual.match(str(valor))) for valor in valores_amostra):
            print(f"\nA coluna '{coluna}' contém valores no formato percentual.")
            colunas_percentuais.append(coluna)

        if any(bool(regex_monetario.match(str(valor))) for valor in valores_amostra):
            print(f"\nA coluna '{coluna}' contém valores no formato monetário.\n")
            colunas_monetarias.append(coluna)

    # Aplica a conversão para valores percentuais
    for coluna in colunas_percentuais:
        df = df.withColumn(
            coluna,
            when(
                col(coluna).rlike("^\d+%$"),
                (regexp_replace(col(coluna), "%", "").cast(DoubleType()) / 100)
            ).otherwise(col(coluna))
        ).withColumn(coluna, col(coluna).cast(DoubleType()))

        print(f"STATUS OK - Coluna {coluna} convertida com sucesso para tipo 'double'.")

    # Aplica a conversão para valores monetários
    for coluna in colunas_monetarias:
        df = df.withColumn(
            coluna,
            when(
                col(coluna).rlike("^R\\$?\\s?\\d{1,3}(\\.\\d{3})*(,\\d{2})?$"),
                regexp_replace(
                    regexp_replace(
                        regexp_replace(col(coluna), "R\\$", ""), 
                        "\\.", "" 
                    ),
                    ",", "."  
                ).cast(DoubleType())
            ).otherwise(col(coluna))
        ).withColumn(coluna, col(coluna).cast(DoubleType()))

        print(f"STATUS OK - Coluna {coluna} convertida com sucesso para tipo 'double'.")

    return df

df_numeric = define_numeric_columns(df_parcelado)

In [0]:
def replace_nulls_with_zero(df):
    
    print(f"Iniciando preenchimentod e valores nulos em colunas inteiros/decimais")
    
    # Identificar colunas numéricas (inteiras e decimais)
    numeric_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, (LongType, DoubleType))]

    print(f"\nColunas numéricas identificadas: {numeric_cols}")

    # Contar valores nulos antes da transformação
    null_counts_before = df.select([count(when(col(c).isNull(), c)).alias(c) for c in numeric_cols]).collect()[0].asDict()
    print(f"\nValores nulos antes da transformação: {null_counts_before}")

    # Substituir valores nulos por 0 nas colunas numéricas
    for col_name in numeric_cols:
        df = df.withColumn(col_name, when(col(col_name).isNull(), 0).otherwise(col(col_name)))
        print(f"\nValor nulo na coluna {col_name} alterado para 0")

    # Contar valores nulos depois da transformação
    null_counts_after = df.select([count(when(col(c).isNull(), c)).alias(c) for c in numeric_cols]).collect()[0].asDict()
    print(f"\nValores nulos depois da transformação: {null_counts_after}\n")

    # Verificar se todas as colunas tiveram seus valores nulos substituídos
    for col_name in numeric_cols:
        if null_counts_after[col_name] == 0:
            print(f"STATUS OK - Coluna {col_name} foi corretamente preenchida.")
        else:
            print(f"STATUS ALERTA -Coluna {col_name} ainda contém valores nulos!")

    return df

df_no_null_numeric = replace_nulls_with_zero(df_numeric)

In [0]:
def replace_nulls_with_hyphen(df):

    print(f"Iniciando preenchimento de valores nulos em colunas strings")

    # Identificar colunas numéricas (inteiras e decimais)
    string_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, (StringType))]

    print(f"\nColunas numéricas identificadas: {string_cols}")

    # Contar valores nulos antes da transformação
    null_counts_before = df.select([count(when(col(c).isNull(), c)).alias(c) for c in string_cols]).collect()[0].asDict()
    print(f"\nValores nulos antes da transformação: {null_counts_before}")

    # Substituir valores nulos por 0 nas colunas numéricas
    for col_name in string_cols:
        df = df.withColumn(col_name, when(col(col_name).isNull(), '-').otherwise(col(col_name)))
        print(f"\nValor nulo na coluna {col_name} alterado para '-'")

    # Contar valores nulos depois da transformação
    null_counts_after = df.select([count(when(col(c).isNull(), c)).alias(c) for c in string_cols]).collect()[0].asDict()
    print(f"\nValores nulos depois da transformação: {null_counts_after}\n")

    # Verificar se todas as colunas tiveram seus valores nulos substituídos
    for col_name in string_cols:
        if null_counts_after[col_name] == 0:
            print(f"STATUS OK - Coluna {col_name} foi corretamente preenchida.")
        else:
            print(f"STATUS ALERTA - Coluna {col_name} ainda contém valores nulos!")

    return df

df_no_null_string = replace_nulls_with_hyphen(df_no_null_numeric)

In [0]:
def extract_memory(df, column_name):

    print(f"Iniciando extração de infromações referente a memoria do console\n")
    
    def extract_memory_info(info):

        if isinstance(info, str) and info:
            padrao = r'(\d+)\s*(G[Bb])'
            resultado = re.search(padrao, info, re.IGNORECASE)
            if resultado:
                return resultado.group(0)
        return '-'

    extrair_memoria_udf = udf(extract_memory_info, StringType())

    df = df.withColumn('memoria', extrair_memoria_udf(col(column_name)))

    if "memoria" in df.columns:
        print("STATUS OK - A coluna 'memoria' foi criada com sucesso.")
    else:
        print("STATUS ALERTA - A coluna 'memoria' não foi encontrada no DataFrame. ❌")


    padrao_gb = r'^\d+\s*[Gg][Bb]$'

    # Conta os registros na coluna 'memoria' que correspondem ao padrão 'XGB' ou 'X GB'
    valid_gb_format_count = df.filter(
        (col("memoria").isNotNull()) &
        (col("memoria") != "-") &
        (regexp_extract(col("memoria"), padrao_gb, 0) != "")
    ).count()

    # Conta o total de linhas no DataFrame
    total_rows = df.count()

    print(f"\nTotal de linhas no DataFrame: {total_rows}")
    print(f"Número de registros na coluna 'memoria' com formato 'XGB' ou 'X GB': {valid_gb_format_count}")

    if valid_gb_format_count > 0:
        print("STATUS OK - A coluna 'memoria' contém valores no formato esperado ('XGB', 'X Gb', 'X gb', etc.)")
        # Opcional: Mostrar algumas linhas para inspecionar os valores
        df.filter(regexp_extract(col("memoria"), padrao_gb, 0) != "").select("nome", "memoria").show(5, truncate=False)
    else:
        print("STATUS ALERTA - A coluna 'memoria' não contém valores no formato esperado.")
        print("Verifique a função 'extract_memory' ou os dados de entrada.")

    return df

df_memory_list = extract_memory(df_no_null_string, 'nome')

In [0]:
def condition_like(df, new_column_name, condition_column, pattern):
    
    
    df = df.withColumn(new_column_name, when(col(condition_column).rlike(pattern), 'Sim').otherwise('Nao'))

    return df

df_type = condition_like(df_memory_list, 'oled', 'nome', '(?i)Oled')
df_type = condition_like(df_type, 'lite', 'nome', '(?i)Lite')

In [0]:
def carregando_tabela_silver(df,delta_table_path):

    print(f"Iniciando o salvamento do DataFrame no formato Delta em: {delta_table_path}")

    try:
        # 1- Obter a contagem de linhas ANTES de salvar ---
        num_rows_to_save = df.count()
        print(f"Número de linhas no DataFrame a ser salvo: {num_rows_to_save}")

        # 2- Salvar o DataFrame no formato Delta ---
        df.write \
                        .format("delta") \
                        .mode("overwrite") \
                        .partitionBy("data_ref") \
                        .save(delta_table_path)

        print(f"DataFrame salvo com sucesso como tabela Delta particionada por 'extract' em: {delta_table_path}")

        # Início das Verificações de Qualidade Pós-Gravação 

        # 3- Garantir que os dados foram salvos no caminho
        print(f"\n--- Verificação: Leitura da Tabela Delta Salva ---")
        df_delta_read = spark.read.format("delta").load(delta_table_path)
        print("Esquema da tabela Delta lida:")
        df_delta_read.printSchema()
        num_rows_saved = df_delta_read.count()

        if df_delta_read.isEmpty():
            print(f"STATUS ALERTA - A tabela Delta salva em '{delta_table_path}' está vazia ou não pôde ser lida.")
        else:
            print(f"STATUS OK - A tabela Delta foi lida com sucesso de '{delta_table_path}' com {num_rows_saved} linhas recarregadas.")


    except Exception as e:
        print(f"Ocorreu um erro geral ao salvar ou verificar a tabela Delta: {e}")


# Caminho para a external location do diretório silver
silver_path = f"/Volumes/nintendodatabrickspkjgt7_workspace/nintendo/silver"

# Sobreescrevendo dados particionados no diretório silver
carregando_tabela_silver(df_type, silver_path)